# Non-linear Breit-Wheeler pair creation

The pair production rate can be written as follows:

$$ 
\frac{d^2N_{BW}}{d\chi_\pm dt} =  \frac{\alpha_f m_e^2 c^4}{\pi \sqrt{3} \hbar \epsilon_\gamma \chi_\gamma} 
\int_x^{+\infty} \sqrt{s} K_{1/3}\left({\frac{2}{3} s^{3/2}}\right) ds - \left({ 2 - \chi_\gamma x^{3/2} }\right) K_{2/3} \left({\frac{2}{3} x^{3/2}}\right) 
$$

where $x = \left( \chi_\gamma / (\chi_- \chi_+) \right)^{2/3}$.

When I ask sympy to peform the integral, it remains stuck forever...

In [2]:
from sympy import *
s = Symbol('s')
func = besselk(1/3, 2/3 * s**(3/2))*sqrt(s)
#print(integrate(func, s))Don't do that

However Wolfram Alpha provides an explicit form for the solution:

$$ 
\int \sqrt{s} K_{1/3}\left({\frac{2}{3} s^{3/2}}\right) ds = \\
\frac{
3^{5/6} \pi ~ s ~ \Gamma\left({\frac{1}{3}}\right) \Gamma\left({\frac{5}{3}}\right) 
\phantom{f}_1F_2\left({ \frac{1}{3}; \frac{2}{3}, \frac{4}{3}; \frac{s^3}{9}}\right)
-
3^{1/6} \pi ~ s^2 ~ \Gamma\left({\frac{2}{3}}\right)^2 
\phantom{f}_1F_2\left({ \frac{2}{3}; \frac{4}{3}, \frac{5}{3}; \frac{s^3}{9}}\right)
}
{9 \Gamma\left({\frac{2}{3}}\right) \Gamma\left({\frac{4}{3}}\right) \Gamma\left({\frac{5}{3}}\right)}
$$

If we find a good open source implementation of generalized hypergeometric functions, we could exploit this in the code.

Anyway, this notebook contains implementations of the pair production rate functions, in order to perform quick test of the library. Let's start with some useful definitions (for code units we use same normalization as smilei):

In [3]:
import math as m
import numpy as np

electron_mass = 9.10938356e-31
elementary_charge = 1.6021766208e-19
speed_of_light = 299792458
reduced_plank = 1.054571800e-34
vacuum_permittivity =  8.854187817e-12
fine_structure_constant =  0.0072973525664

classical_elec_radius = (1./4./np.pi/vacuum_permittivity)*( elementary_charge**2 / (electron_mass * speed_of_light**2))

lambda_laser = 0.8e-6

schwinger_field_SI = electron_mass**2 * speed_of_light**3 / (reduced_plank*elementary_charge)
print('Schiwinger field in SI units is: ' + str(schwinger_field_SI))

schwinger_field_code_units = electron_mass*speed_of_light*lambda_laser/(2.0*reduced_plank*m.pi)
print('Schiwinger field in code units is: ' + str(schwinger_field_code_units))

conversion_of_rate_from_code_to_SI = (2.0 * m.pi * speed_of_light)/lambda_laser
print('Conv. factors for prod. rate from code units to SI: ' + str(conversion_of_rate_from_code_to_SI))

conversion_of_rate_from_SI_to_code = 1.0/conversion_of_rate_from_code_to_SI
print('Conv. factors for prod. rate from SI to code units: ' + str(conversion_of_rate_from_SI_to_code))

field_reference_PIC = 2.0 * np.pi * electron_mass * speed_of_light * speed_of_light / (elementary_charge*lambda_laser)
print('Field reference PIC: {:e}'.format(field_reference_PIC))

Schiwinger field in SI units is: 1.3232854660718482e+18
Schiwinger field in code units is: 329718.7590461917
Conv. factors for prod. rate from code units to SI: 2354564459136066.5
Conv. factors for prod. rate from SI to code units: 4.247069967100916e-16
Field reference PIC: 4.013376e+12


Let's define the $\chi_\gamma $ function. We will assume that all these quantities are provided in code units (same normalization as Smilei)

In [4]:
def calc_chi_gamma(p, E, B):
    gamma_phot = np.linalg.norm(p)  
    c = p/gamma_phot  
    loc_field = gamma_phot * np.linalg.norm( E - np.dot(c,E)*c + np.cross(c,B))     
    return loc_field/schwinger_field_code_units

#A quick tests
a = calc_chi_gamma(np.array([83.759, 139.311, -230.553]), np.array([-166.145, -78.231, -278.856]), np.array([-279.174, -158.849, -93.826]))

b = calc_chi_gamma(np.array([-201.076, -170.956, 66.094]), np.array([-201.968, -114.862, 3.580]), np.array([-195.188, 223.384, -204.352]))

c = calc_chi_gamma(np.array([-219.959, 135.362, 135.086]), np.array([110.184, 272.527, 64.938]), np.array([-127.926, -21.659, -59.226]))

d = calc_chi_gamma(np.array([-2314.45, -2356.30, 546.28]), np.array([1230.11, 1638.02, -2911.04]), np.array([-2203.66, 1243.79, -2830.99]))

e = calc_chi_gamma(np.array([9.2627, -25.4575, -10.2246]), np.array([2.9271, 10.4293, 3.6103]), np.array([1.7439, 1.9778, 17.8799]))



print(a)
print(b)
print(c)
print(d)
print(e)


0.347111844317
0.315109658214
0.200401320634
57.2204397969
0.000904147405336


We now define 

$$
inner(x) = \int_x^{+\infty} \sqrt{s} K_{1/3}\left({\frac{2}{3} s^{3/2}}\right) ds
$$

and 

$$
X(\chi_\gamma, \chi_-) = \left( \frac{\chi_\gamma}{\chi_- (\chi_\gamma - \chi_- )}   \right)^{2/3}
$$

and

$$
T(\chi_\gamma) = \frac{1}{\pi \sqrt{3} \chi^2_\gamma} \int_0^{+\infty} inner\left({X(\chi_\gamma, \chi_-)}\right) - \left({ 2 - \chi_\gamma X(\chi_\gamma, \chi_-)^{3/2} }\right) K_{2/3} \left({\frac{2}{3} X(\chi_\gamma, \chi_-)^{3/2}}\right) d\chi_-
$$

In [5]:
import scipy.special as spe
import scipy.integrate as integ

def inner(x):
    return integ.quad(lambda s: np.sqrt(s)*spe.kv(1./3., 2./3. * s**(3./2.)), x, np.inf)[0] 

inner = np.vectorize(inner)

def X(chi_phot, chi_ele):
    if (chi_phot > chi_ele and chi_ele != 0):
        return np.power(chi_phot/(chi_ele*(chi_phot-chi_ele)), 2./3.)        
    else:
        return 1.0e30

X = np.vectorize(X)

def T(chi_phot):
    coeff = 1./(np.pi * np.sqrt(3.) * chi_phot * chi_phot)
    return integ.quad(lambda chi_ele: 
                      coeff*(inner(X(chi_phot, chi_ele)) - 
                      (2.0 - chi_phot*np.power(X(chi_phot, chi_ele), 3./2.))*spe.kv(2./3., 2./3. *X(chi_phot, chi_ele)**(3./2.)) )
                      , 0, chi_phot)[0] 
T = np.vectorize(T)
    

We can now write the total production rate as:

$$
\frac{dN_{BW}}{dt} = \frac{\alpha_f m_e^2 c^4}{\hbar \epsilon_\gamma} \chi_\gamma T(\chi_\gamma)
$$

we also note that $\epsilon_\gamma / m_e c^2$ is the  photon energy in code units.
So we can write the following function, which gives the total production rate *in SI units* as a function of quantities *in code units*. 

In [6]:
def dNBW_dt(chi_phot, e_phot):
    val = ((electron_mass*(speed_of_light)**2)*fine_structure_constant/reduced_plank)*(chi_phot/e_phot)*T(chi_phot)
    return val

px = 9000
chi = calc_chi_gamma(np.array([px,0,0]),np.array([0,500,0]),np.array([0,0,0]))
rate = dNBW_dt(calc_chi_gamma(np.array([px,0,0]),np.array([0,500,0]),np.array([0,0,0])), px)
print(str(chi) + " " + str(rate) + " " + str(rate/1e15))

p = np.array([61019.1, -24359.3, 65116.2])
E = np.array([69942.0, 38024.7, -43604.1])
B = np.array([-26990.0, 58267.8, -63485.8])
print(dNBW_dt(np.asscalar(calc_chi_gamma(p, E, B)), np.asscalar(np.linalg.norm(p)))*conversion_of_rate_from_SI_to_code)

p = np.array([-965.61, -3975.11, 6917.22])
E = np.array([11.17, -2117.72, -1407.19])
B = np.array([6259.79, 7557.54, 773.11])
print(dNBW_dt(np.asscalar(calc_chi_gamma(p, E, B)), np.asscalar(np.linalg.norm(p)))*conversion_of_rate_from_SI_to_code)
p = np.array([149.825, 933.115, -538.195])
E = np.array([931.686, -861.074, 944.652])
B = np.array([531.406, 670.933, 660.057])
print(dNBW_dt(np.asscalar(calc_chi_gamma(p, E, B)), np.asscalar(np.linalg.norm(p)))*conversion_of_rate_from_SI_to_code)
p = np.array([-44.4546, -0.2033, 94.5843])
E = np.array([39.8996, -29.2501, 58.7720])
B = np.array([44.3417, 15.5024, 29.4024])
print(dNBW_dt(np.asscalar(calc_chi_gamma(p, E, B)), np.asscalar(np.linalg.norm(p)))*conversion_of_rate_from_SI_to_code)
p = np.array([6.81696,9.68933,2.81229])
E = np.array([-4.89986,-9.65535,3.69471])
B = np.array([8.89549,-5.46574,-6.75393])
print(dNBW_dt(np.asscalar(calc_chi_gamma(p, E, B)), np.asscalar(np.linalg.norm(p)))*conversion_of_rate_from_SI_to_code)
p = np.array([-2552.37,3970.67,-62.9812])
E = np.array([800,800,800])
B = np.array([800,800,800])
print("chi:" + str(calc_chi_gamma(p, E, B)))
print(dNBW_dt(np.asscalar(calc_chi_gamma(p, E, B)), np.asscalar(np.linalg.norm(p)))*conversion_of_rate_from_SI_to_code)

13.6479950762 9.32177159432e+14 0.932177159432
7.63488202211
3.51855878777
1.50648551484
4.69766211952e-73
0.0
chi:27.667906063
1.41557597076


In [7]:
px = np.logspace(-1, 8.0, num=100, base=10)

def ff(px):
    return dNBW_dt(calc_chi_gamma(np.array([px,0,0]),np.array([0,500,0]),np.array([0,0,0])), px)

ff = np.vectorize(ff)

def cc(px):
    return calc_chi_gamma(np.array([px,0,0]),np.array([0,500,0]),np.array([0,0,0]))

cc = np.vectorize(cc)

ff(px)

import matplotlib.pyplot as plt

plt.loglog(cc(px), ff(px)*1e-15)
axes = plt.gca()
axes.set_xlim([0.1,100000])
axes.set_ylim([0.00001,1])
plt.show()

# Quantum synchrotron emission

In [8]:
def calc_chi_elec(p, E, B):
    gamma_elec = sqrt(1.0 + np.dot(p,p))  
    v = p/gamma_elec  
    evcb = E + np.cross(v,B)
    loc_field = gamma_elec * sqrt(abs(np.dot(v,E)**2 - np.dot(evcb,evcb)))     
    return loc_field/schwinger_field_code_units

calc_chi_elec(np.array([1000, 0, 0]), np.array([0, 0, 0]), np.array([0, 0, 500]))

1.51644389735785

In [15]:
chi_phot_min = 0.01

def comp_y(chi_phot, chi_part):
    if(chi_part == chi_phot):
        return 0.0
    return chi_phot/(3*chi_part*(chi_part-chi_phot))

def inner_fun(y):
    return integ.quad(lambda s: spe.kv(1./3.,s), 2*y, np.inf)[0] 

def KK_int(chi_part, chi_phot):
    yy = comp_y(chi_phot, chi_part)
    inner = inner_fun(yy)
    part_2 = spe.kv(2./3.,2*yy)*(2 + 3*chi_phot*yy)
    return ( part_2 - inner)/(pi*sqrt(3))
    
def KK_func(chi_part):
    return integ.quad(lambda s: KK_int(chi_part,s), chi_phot_min, chi_part)[0] 

tau_e = classical_elec_radius/speed_of_light

def qs_dn_dt(chi_part, gamma_part):
    return (fine_structure_constant/tau_e)*KK_func(chi_part)/(chi_part*gamma_part)

print(KK_func(0.001))
print(KK_func(0.01))
print(KK_func(0.1))
print(KK_func(1))
print(KK_func(10.))
print(KK_func(100.))
print(KK_func(10000.))

/usr/lib/python3/dist-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


nan
nan
0.0013032229454470994
0.7032369286240776
53.93181891450526
2912.9421286618954
6698656.806401038


# Miscellanea

In [9]:
from decimal import Decimal

I = 1e24
I_W_m2 = I * 1e4 
E = sqrt(I_W_m2 * 2 / speed_of_light / vacuum_permittivity)
print("{:6.3e}".format(E))
print("{:6.3e}".format(E/field_reference_PIC))
print("{:6.3e}".format(E/schwinger_field_SI))

B = 2500000
E = B*speed_of_light/speed_of_light
calc_chi_elec(np.array([1800, 0, 0]), np.array([0, 0, 0]), np.array([0, 0, E/field_reference_PIC]))

2.745e+15
6.839e+2
2.074e-3


3.40062678490544e-9

In [10]:
a = calc_chi_elec(np.array([24.3752, -11.5710, -10.0841]), np.array([57.185, -16.6555, 22.4340]), np.array([6.6911, -23.8724, 13.9934]))
b = calc_chi_elec(np.array([4.015, 197.287, 141.705]), np.array([30.287, 115.740, 120.891]), np.array([-190.161, -129.115, -57.002]))
c = calc_chi_elec(np.array([-2534.83, 1011.54, -793.04]), np.array([741.67, -2359.97, 1463.50]), np.array([1477.19, -1448.33, 1953.68]))

print(a)
print(b)
print(c)

0.00216716627219670
0.166318112874468
16.0114572646993
